### Preparing the R environment

Install the required packages to be used in the project, set the themes for the plot and configure warning suppresion for cleaner output.

In [12]:
# Suppressing warnings for a cleaner output
options(warn=-1)

# Installing required packages for plotting and training
library(ggpubr)
library(tidymodels)

# Setting up the plot size and theme
options(repr.plot.width = 14, repr.plot.height = 6)

theme <-
  theme_minimal() +
  theme(
    plot.title = element_text(size=18, face="bold", color="gray30"),
    plot.subtitle = element_text(size=15, face="bold", color="gray30"),
    axis.title = element_text(size=15, face="bold", color="gray30"),
    plot.caption = element_text(size=15, face="bold", color = "gray60"),
    legend.text = element_text(size=13, color="gray30"),
    legend.title = element_text(size=13, color="gray30"),
    strip.text.x = element_text(size=13, face="bold", color="gray30")
  )

theme_set(theme)

Loading required package: ggplot2
-- Attaching packages -------------------------------------- tidymodels 0.1.2 --
v broom     0.7.2      v recipes   0.1.15
v dials     0.0.9      v rsample   0.0.8 
v dplyr     1.0.2      v tibble    3.0.4 
v infer     0.5.3      v tidyr     1.1.2 
v modeldata 0.1.0      v tune      0.1.2 
v parsnip   0.1.4      v workflows 0.2.1 
v purrr     0.3.4      v yardstick 0.0.7 
-- Conflicts ----------------------------------------- tidymodels_conflicts() --
x purrr::discard() masks scales::discard()
x dplyr::filter()  masks stats::filter()
x dplyr::lag()     masks stats::lag()
x recipes::step()  masks stats::step()


# Exploratory Data Analysis: COVID-19

In this Jupyter notebook, we perform a self expertimentation of Exploratory Data Analysis (EDA) for Denmark's COVID-19 cases in R. R is a language and environment for statistical computing and graphics ([*source: R project*](https://www.r-project.org/about.html)) which fits perfectly for this project as the goal is to visualize data and tell a conclusion/story based on real decisions.

## 1. Identify business problem

The first step to Exploratory Data Analysis (EDA) is to identify the business problem. The business case revolves around the understanding of the situation and the use of data to prevent spread of COVID-19 virus in Denmark. The aim is to analyze the data and compare against the decisions made and enforced by the officials and the health experts, primarily the two lockdowns from March and Decemberof 2020. This EDA is not a solution to the problem but the ambition is to understand if gathering factual data aided in effective decision making by the authorities. 

## 2. Data collection

The data used in this analysis is collected from Statistibanken, which is a public database maintained and made available by Danmarks Statistik --- the central statistics office in Denmark under the Ministry of Economy. The data consists of official records of total people tested, infections, deaths and admission to hospital to among other key figures. The data was collected starting from March 21, 2020, which gets updated frequently. The latest data as of conducting the experiment is up until December 15, 2020.

## 3. Data preparation

The data from Statistikbanken is made available in multiple formats so people can choose how they want the data, for example: an Excel sheet or a plain text file. For this analaysis, we chose to use the data as a CSV (comma-separated value) file for ease of use. In the following steps, we will go through the process of loading the data, tidying it and preparing it for the analysis.

### 3.1 Load the dataset

In the following steps, we load the dataset into this R notebook and inspect if it is workable and understandable. If not, we must proceed to transform the data as deemed necessary. Without appropraite and clear understanding of data, the anaylsis will be out of scope.

After we load the data into the R notebook, we examine to ensure the target data and/or key figures exists. 

In [13]:
list.files(".")

[1] "covid-data-statistikbanken.csv" "EDA-COVID.ipynb"               
[3] "README.md"

In [14]:
# Create a dataframe by reading the file containing the dataset

covid_data_sb <- read.csv("covid-data-statistikbanken.csv")
head(covid_data_sb)

X.                                               M03D21 M03D22 M03D23 M03D24
1 Confirmed cases of COVID-19 (total)              1326   1395   1450   1577  
2 Recovered from COVID-19 infection (total         ..     ..     ..     ..    
3 Number of persons tested for COVID-19 (total)    11657  12351  12843  13756 
4 Deaths with COVID-19 (total)                     22     25     31     37    
5 Deaths with COVID-19 per day                     3      3      6      6     
6 Persons hospitalised in connection with COVID-19 206    232    254    301   
  M03D25 M03D26 M03D27 M03D28 M03D29 ... M12D06  M12D07  M12D08  M12D09 
1 1715   1851   2010   2201   2395   ...   90603   92649   94799   97357
2 ..     ..     ..     ..     ..     ...   70876   71882   73052   74367
3 14870  15981  17275  18810  20198  ... 3524861 3541148 3556264 3572954
4 43     55     67     76     84     ...     899     905     912     922
5 6      12     12     9      8      ...       8       6       7      10
6 350    386    430    459    499    ...     302     328     354     349
  M12D10  M12D11  M12D12  M12D13  M12D14  M12D15 
1  100489  103564  107116  109758  113095  116087
2   75664   76927   78236   79394   80564   82099
3 3592677 3611456 3630815 3646152 3664247 3680607
4     928     935     943     954     961     961
5       6       7       8      11       7       0
6     342     361     371     390     429     439

### 3.2 Data wrangling

Now that the target data is loaded and ready to be used, we print a few columns of the data with `head` command and inspect if any transformations are required. In the initial inspection, the data looks to be well prepared and organized but still not ready to be used. Based on the requirements and researchers, the transformations required is subjective. For our use case, the required transformations are listed below:

_**This process of transforming data into another format is called "data wrangling".**_

1. The key indicators, i.e the names describing the values, are the first values of each rows. Ideally, they should be the first values in each of columns. In simple terms, the x-axis and y-axis needs to be reversed. This requires the dataframe to be transposed (or rotated).
2. The key indicator names are very long. To adhere to R's naming covention, we rename each of the key indicators to short, meaningful names and follow snake case using "_", for example `some_key_indicator`.
3. The format of the date is "MxxDxx". For maximum flexibility and understanding, we transform this to the international ISO date format of `yyyy-mm-dd`. This is done using R's built in functions in the following steps.
4. This dataset is missing one of the key figures, which is the daily new cases. Since we have the total number of cases for everyday, we substract that value of a day with that value of the day before to calculate the daily new cases. 
    - For example: total case for March 22 = 1395, total case for March 21 = 1326. So daily new case of March 21 = 1395 - 1326 = 69.
5. The missing values are represented by `..`. Those should updated to either empty values or `NA` so that R can interpret it as missing value.
6. Change the data types of the attributes if needed. Since the data is read from a CSV file, R may not be able to correctly interpret the data type. For example, if `total_tested` is interpreted as `character` instead of `numeric`, it'll fail to plot the data or train a model based on that data.

In [15]:
# 1. Transposing a dataframe: https://stackoverflow.com/a/6645587/9332260
denmark_covid_data <- t(covid_data_sb[,2:ncol(covid_data_sb)])
colnames(denmark_covid_data) <- covid_data_sb[,1] 
head(denmark_covid_data)

Confirmed cases of COVID-19 (total)
M03D21 1326                               
M03D22 1395                               
M03D23 1450                               
M03D24 1577                               
M03D25 1715                               
M03D26 1851                               
       Recovered from COVID-19 infection (total
M03D21 ..                                      
M03D22 ..                                      
M03D23 ..                                      
M03D24 ..                                      
M03D25 ..                                      
M03D26 ..                                      
       Number of persons tested for COVID-19 (total)
M03D21 11657                                        
M03D22 12351                                        
M03D23 12843                                        
M03D24 13756                                        
M03D25 14870                                        
M03D26 15981                                        
       Deaths with COVID-19 (total) Deaths with COVID-19 per day
M03D21 22                           3                           
M03D22 25                           3                           
M03D23 31                           6                           
M03D24 37                           6                           
M03D25 43                           6                           
M03D26 55                           12                          
       Persons hospitalised in connection with COVID-19
M03D21 206                                             
M03D22 232                                             
M03D23 254                                             
M03D24 301                                             
M03D25 350                                             
M03D26 386                                             
       Patients in intensive care units in connection with COVID-19
M03D21 42                                                          
M03D22 46                                                          
M03D23 55                                                          
M03D24 59                                                          
M03D25 87                                                          
M03D26 94                                                          
       Patients on ventilators in connection with COVID-19
M03D21 35                                                 
M03D22 40                                                 
M03D23 47                                                 
M03D24 58                                                 
M03D25 76                                                 
M03D26 78

In [16]:
# 2. Rename column names: https://stackoverflow.com/a/16031013/9332260
denmark_covid_data <- cbind(rownames(denmark_covid_data), denmark_covid_data)
rownames(denmark_covid_data) <- NULL
colnames(denmark_covid_data) <- c("original_date",
                                  "total_cases",
                                  "total_recovered",
                                  "total_tested",
                                  "total_deaths",
                                  "daily_deaths",
                                  "daily_hospitalized",
                                  "daily_patients_icu",
                                  "daily_patients_ventilators")
denmark_covid_data <- as.data.frame(denmark_covid_data)

In [17]:
# Verifying that the renaming of columns was successful
head(denmark_covid_data)

original_date total_cases total_recovered total_tested total_deaths
1 M03D21        1326        ..              11657        22          
2 M03D22        1395        ..              12351        25          
3 M03D23        1450        ..              12843        31          
4 M03D24        1577        ..              13756        37          
5 M03D25        1715        ..              14870        43          
6 M03D26        1851        ..              15981        55          
  daily_deaths daily_hospitalized daily_patients_icu daily_patients_ventilators
1 3            206                42                 35                        
2 3            232                46                 40                        
3 6            254                55                 47                        
4 6            301                59                 58                        
5 6            350                87                 76                        
6 12           386                94                 78

In [18]:
# 3. Change date format to ISO format
# 5. Change missing data ".." to NA 
denmark_covid_data <- denmark_covid_data %>%
  mutate(
      date = as.Date(original_date, "M%mD%d"),
      day = row_number(date) - 1,
      total_recovered = na_if(total_recovered, ".."),
      total_tested = na_if(total_tested, ".."),
      daily_hospitalized = na_if(daily_hospitalized, ".."),
      daily_patients_icu = na_if(daily_patients_icu, ".."),
      daily_patients_ventilators = na_if(daily_patients_ventilators, ".."),
  )

In [24]:
# Inspect the structure of the dataframe
str(denmark_covid_data)

'data.frame':	270 obs. of  12 variables:
 $ original_date             : Factor w/ 270 levels "M03D21","M03D22",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ total_cases               : num  1326 1395 1450 1577 1715 ...
 $ total_recovered           : num  NA NA NA NA NA NA NA NA NA NA ...
 $ total_tested              : num  11657 12351 12843 13756 14870 ...
 $ total_deaths              : num  22 25 31 37 43 55 67 76 84 95 ...
 $ daily_deaths              : num  3 3 6 6 6 12 12 9 8 11 ...
 $ daily_hospitalized        : num  206 232 254 301 350 386 430 459 499 533 ...
 $ daily_patients_icu        : num  42 46 55 59 87 94 109 121 131 137 ...
 $ daily_patients_ventilators: num  35 40 47 58 76 78 89 104 113 119 ...
 $ date                      : Date, format: "2020-03-21" "2020-03-22" ...
 $ day                       : num  0 1 2 3 4 5 6 7 8 9 ...
 $ new_cases                 : num  71 69 55 127 138 136 159 191 194 160 ...


In [20]:
# 6. Update the data types 
denmark_covid_data$total_cases <- as.numeric(as.character(denmark_covid_data$total_cases))
denmark_covid_data$total_recovered <- as.numeric(as.character(denmark_covid_data$total_recovered))
denmark_covid_data$total_tested <- as.numeric(as.character(denmark_covid_data$total_tested))
denmark_covid_data$total_deaths <- as.numeric(as.character(denmark_covid_data$total_deaths))
denmark_covid_data$daily_deaths <- as.numeric(as.character(denmark_covid_data$daily_deaths))
denmark_covid_data$daily_hospitalized <- as.numeric(as.character(denmark_covid_data$daily_hospitalized))
denmark_covid_data$daily_patients_icu <- as.numeric(as.character(denmark_covid_data$daily_patients_icu))
denmark_covid_data$daily_patients_ventilators <- as.numeric(as.character(denmark_covid_data$daily_patients_ventilators))

In [22]:
# 5. Add a new column for daily new cases
denmark_covid_data <- 
denmark_covid_data %>%
  arrange(date) %>%
   mutate(
       new_cases = total_cases - lag(total_cases, 1, 0),
   ) %>% arrange(date)

denmark_covid_data$new_cases[1]=71

In [23]:
denmark_covid_data

original_date total_cases total_recovered total_tested total_deaths
1   M03D21        1326          NA            11657         22         
2   M03D22        1395          NA            12351         25         
3   M03D23        1450          NA            12843         31         
4   M03D24        1577          NA            13756         37         
5   M03D25        1715          NA            14870         43         
6   M03D26        1851          NA            15981         55         
7   M03D27        2010          NA            17275         67         
8   M03D28        2201          NA            18810         76         
9   M03D29        2395          NA            20198         84         
10  M03D30        2555          NA            21378         95         
11  M03D31        2815          NA            23763        110         
12  M04D01        3092          NA            26776        132         
13  M04D02        3355        1089            29921        150         
14  M04D03        3672        1193            34388        170         
15  M04D04        4077        1283            39928        185         
16  M04D05        4369        1327            43734        197         
17  M04D06        4647        1378            46916        214         
18  M04D07        4978        1491            51544        226         
19  M04D08        5386        1621            57535        241         
20  M04D09        5635        1736            62210        254         
21  M04D10        5819        1773            64731        270         
22  M04D11        5996        1955            67771        281         
23  M04D12        6174        2123            70125        292         
24  M04D13        6318        2235            72099        308         
25  M04D14        6496        2515            73919        320         
26  M04D15        6681        2748            77712        330         
27  M04D16        6879        3023            82380        342         
28  M04D17        7073        3389            87024        351         
29  M04D18        7242        3847            91437        361         
30  M04D19        7384        4141            94277        367         
... ...           ...         ...             ...          ...         
241 M11D16         63331      48696           3181467      769         
242 M11D17         64551      49961           3199098      772         
243 M11D18         65808      51217           3220265      777         
244 M11D19         67105      52445           3239806      780         
245 M11D20         68362      53470           3258149      784         
246 M11D21         69635      54436           3275193      789         
247 M11D22         70485      55274           3289511      795         
248 M11D23         71654      56032           3308512      798         
249 M11D24         73021      57075           3329222      804         
250 M11D25         74204      58148           3346932      812         
251 M11D26         75395      59250           3364827      816         
252 M11D27         76718      60312           3383120      824         
253 M11D28         78354      61461           3404692      830         
254 M11D29         79352      62514           3418006      841         
255 M11D30         80481      63515           3432519      848         
256 M12D01         81494      64757           3448151      858         
257 M12D02         83535      66086           3464269      864         
258 M12D03         85140      67416           3478986      874         
259 M12D04         86743      68632           3493655      881         
260 M12D05         88858      69869           3510327      891         
261 M12D06         90603      70876           3524861      899         
262 M12D07         92649      71882           3541148      905         
263 M12D08         94799      73052           3556264      912         
264 M12D09         97357      74367         

### 3.3 Verify data

In the previous iterations of this analysis, we used the dataset from [Our World in Data](https://ourworldindata.org/coronavirus/country/denmark?country=~DNK), who sourced their data from John Hopkins University. During that analysis, we had created plot for total confirmed cases in Denmark which had numbers of around 8.5 million confirmed cases. Denmark has a total population of under 6 million so that dataset had wrong data. To confirm that the issue was not in our code, we also did a manual inspection of the CSV file, and the total confirmed cases was more than 8.5 million.

To ensure we don't face similar issue, manual cross-checking of data from StatisktikBanken was done against the aforementioned Our World in Data dataset from previous iterations, to ensure its validaity and integreity.

## 4. Analyze the data here!!!!!!!!!!!

Now that we have changed the data types of the input data, we can start analyzing the data and gain understanding from it. To plot the data, we use the library `tidyverse` because of its flexibility and customization abilities.

In [ ]:
# Daily cases
# options(repr.plot.width=7, repr.plot.height=5)

ggplot(denmark_covid_data, aes(x = date)) +

geom_rect(aes(                                                             # Rectangular shade for emphasizing on lockdown #1
    xmin = as.Date("2020-03-15"),
    xmax = as.Date("2020-04-13"),
    ymin = -Inf, ymax = Inf, fill = "Lockdown period")) +
geom_rect(aes(                                                             # Rectangular shade for emphasizing on lockdown #2
    xmin = as.Date("2020-12-08"),
    xmax = as.Date("2020-12-18"),
    ymin = -Inf, ymax = Inf, fill = "Lockdown period")) +
geom_vline(aes(                                                            # Vertical dashed line
    xintercept = as.Date("2020-10-29"),
    linetype = "Break"),
           color = "black",
           lwd = 0.7
          ) +


geom_line(aes(y = new_cases, color = "dodgerblue4"), lwd=1) +
scale_x_date(breaks = date_breaks("1 months"), date_labels = "%b") +       # Improving the x-axis to show all months

xlab("\nMonths in 2020") +                                                 # Label for x-axis
scale_y_continuous("No. of people\n") +                                    # Label for y-axis

ggtitle("COVID-19: Daily cases") +                          # Title for the plot
scale_fill_manual(name = "",                                               # Legend for the rectangular shade
                  values="burlywood1",
                  labels=c("Lockdown periods")) +
scale_color_identity(name = "",                                            # Legend for the line plots
                     breaks = c("dodgerblue4"),
                     labels = c("Daily cases"),
                     guide = "legend") +
scale_linetype_manual(name = "",                                           # Legend for the vertical dotted line
                      values = c("Break" = "dashed"),
                      labels = c("Mask requirement"))

### 4.1 Deaths, hospitalized and lockdowns

Starting with one of the most impactful plotting, we analyze the data for the number of deaths and hospitalized. Along with that, we also emphasize on the 2 lockdowns enforced by the state officials, the first one from March 15, 2020 to April 13, 2020 and the recent one starting from December 9, 2020 onwards.

In [ ]:
# options(repr.plot.width=7, repr.plot.height=4.5)

hospitalized_plot <- ggplot(denmark_covid_data, aes(x = date)) +                                # Specify the data frame and x-axis for the plot
geom_rect(aes(                                                             # Rectangular shade for emphasizing on lockdown #1
    xmin = as.Date("2020-03-15"),
    xmax = as.Date("2020-04-13"),
    ymin = -Inf, ymax = Inf, fill = "Lockdown period")) +
geom_rect(aes(                                                             # Rectangular shade for emphasizing on lockdown #2
    xmin = as.Date("2020-12-08"),
    xmax = as.Date("2020-12-15"),
    ymin = -Inf, ymax = Inf, fill = "Lockdown period")) +

geom_vline(aes(                                                            # Vertical dashed line
    xintercept = as.Date("2020-10-29"),
    linetype = "Break"),
           color = "black",
           lwd = 0.7
          ) +

geom_line(aes(y = daily_hospitalized, color = "blue"), lwd = 1) +               # line plot for hospitalized

scale_x_date(breaks = date_breaks("1 months"), date_labels = "%b") +       # Improving the x-axis to show all months

xlab("\nMonths in 2020") +                                                 # Label for x-axis
scale_y_continuous("No. of people") +                                    # Label for y-axis

ggtitle("COVID-19: Hospitalization") +                          # Title for the plot
scale_fill_manual(name = "",                                               # Legend for the rectangular shade
                  values="burlywood1",
                  labels=c("Lockdown periods")) +
scale_color_identity(name = "",                                            # Legend for the line plots
                     breaks = c("blue", "red"),
                     labels = c("Hospitalized", "Deaths"),
                     ) +
scale_linetype_manual(name = "",                                           # Legend for the vertical dotted line
                      values = c("Break" = "dashed"),
                      labels = c("Mask requirement")) 
# multiple_grid_plot_theme_config
# theme(legend.position = c(1, 1), legend.justification = c(1, 1))

deaths_plot <- ggplot(denmark_covid_data, aes(x = date)) +                                # Specify the data frame and x-axis for the plot
geom_rect(aes(                                                             # Rectangular shade for emphasizing on lockdown #1
    xmin = as.Date("2020-03-15"),
    xmax = as.Date("2020-04-13"),
    ymin = -Inf, ymax = Inf, fill = "Lockdown period")) +
geom_rect(aes(                                                             # Rectangular shade for emphasizing on lockdown #2
    xmin = as.Date("2020-12-08"),
    xmax = as.Date("2020-12-15"),
    ymin = -Inf, ymax = Inf, fill = "Lockdown period")) +

geom_vline(aes(                                                            # Vertical dashed line
    xintercept = as.Date("2020-10-29"),
    linetype = "Break"),
           color = "black",
           lwd = 0.7
          ) +

geom_line(aes(y = daily_deaths, color = "red"), lwd = 1) +                 # Line plot for deaths
# geom_line(aes(y = daily_hospitalized, color = "blue"), lwd = 1) +               # line plot for hospitalized

scale_x_date(breaks = date_breaks("1 months"), date_labels = "%b") +       # Improving the x-axis to show all months

xlab("\nMonths in 2020") +                                                 # Label for x-axis
scale_y_continuous("No. of people") +                                    # Label for y-axis

ggtitle("COVID-19: Deaths") +                          # Title for the plot
scale_fill_manual(name = "",                                               # Legend for the rectangular shade
                  values="burlywood1",
                  labels=c("Lockdown periods")) +
scale_color_identity(name = "",                                            # Legend for the line plots
                     breaks = c("red"),
                     labels = c("Deaths")
                     ) +
scale_linetype_manual(name = "",                                           # Legend for the vertical dotted line
                      values = c("Break" = "dashed"),
                      labels = c("Mask requirement"))
# multiple_grid_plot_theme_config

ggarrange(hospitalized_plot, deaths_plot, ncol=2, common.legend = TRUE, legend="bottom") 


After the initial plotting, it is clear that there was a strong impact of COVID-19 around late March--early April. The number of people who were dying due to COVID-19, as indicated by red line, and who were hospitalized, as indicated by the blue line, increased sharply. The first, month-long lockdown was imposed during this period, as indicated by the soft-orange shade. The result of the lockdown was effective as the number of reported hospitalization started to decline and kept declining over the next few months. The number of deaths was still rising at the end of the first lockdown, but started to stabilize soon after.

This was a positive indication that the imposed lockdown was effective in controlling the spread of the virus.

As the restictions started to loosen up, the public started to gather in larger numbers, which was an opportunity for the virus to spread again. This is indicated by the rise on both total deaths and hospitalized starting September. To control the rapid increase, the government mandated the use of mask on all public places starting October 29, represented by the black, dashed line.

Ever since, the numbers started to grow. The plot above shows a sharp incline starting November. And with the holidays starting, the government imposed second lockdown starting December 09, 2020, which is the date of conducting this experiment. This was an attempt to reduce the number of gatherings and parties, and limit them to families to control the spread of the COVID-19.

### 4.2 Total confirmed cased and total tested

While the first plot was impactful, a more meaningful one would be to analyze the data between total number of people tested (plot A) and total number of confirmed cases (plot B). When visualising them individually, both of the plots A and B shows almost linear growth as the months go by. Without much context, this is very worrying and can be taken out of proportion. By when combining them and visualizing them on the same plot (plot C), it becomes much clearer that the situation is not as bad. Yes, the number of cases are increasing, but when comparing them to the number of tests done, the result has a vastly different conclusion.

This is an example for understanding the importance of scope and context while conducting an analysis.

In [ ]:
str(denmark_covid_data)

In [ ]:
##########################################################################################################################
############ Step 1: First we create a plot "A" for total number of people tested and assign it to a variable ############
##########################################################################################################################

total_tested_plot <- ggplot(denmark_covid_data, aes(x = date)) +        # Specify the data frame and x-axis for the plot
geom_line(aes(y = total_tested, color = "red"), lwd = 1) +               # Line plot for total number of peple tested
scale_x_date(breaks = date_breaks("1 months"), date_labels = "%b") +    # Improving the x-axis to show all months
xlab("Months in 2020") +                                                # Label for x-axis
scale_y_continuous("No. of people tested",                              # Label for y-axis
                     labels = function(x) prettyNum(x, big.mark=",", scientific = FALSE)
                   ) +
ggtitle("COVID-19: Total tested") +                                     # Title for the plot A
scale_color_identity(name = "",                                         # Legend for the line plot
                     breaks = "red",
                     labels = "Total tested",
                     guide = "legend"
                     )
# multiple_grid_plot_theme_config


##########################################################################################################################
############Step 2: Then we create a plot "B" for total number of confirmed cases and assign it to a variable ###########
##########################################################################################################################

total_confirmed_cases_plot <- ggplot(denmark_covid_data, aes(x = date)) +   # Specify the data frame and x-axis for the plot
geom_line(aes(y = total_cases, color = "blue"), lwd = 1) +                  # Line plot for total confirmed cases
scale_x_date(breaks = date_breaks("1 months"), date_labels = "%b") +        # Improving the x-axis to show all months
xlab("Months in 2020") +                                                    # Label for x-axis
scale_y_continuous("No. of confirmed cases",                                # Label for y-axis
                     labels = function(x) prettyNum(x, big.mark=",", scientific = FALSE)
                   ) +
ggtitle("COVID-19: Total confimed cases") +                                 # Title for the plot
scale_color_identity(name = "",                                             # Legend for the line plot
                     breaks = "blue",
                     labels = "Total confirmed cases",
                     guide = "legend"
                     )
# multiple_grid_plot_theme_config


##########################################################################################################################
##################### Step 3: Then we create a combined plot for "A" & "B" and assign it to a variable ###################
##########################################################################################################################

combined_plot <- ggplot(denmark_covid_data, aes(x = date)) +             # Specify the data frame and x-axis for the plot
geom_line(aes(y = total_tested, color = "red"), lwd = 1) +                # Line plot for total number of peple tested
geom_line(aes(y = total_cases, color = "blue"), lwd = 1) +               # Line plot for total confirmed cases
scale_x_date(breaks = date_breaks("1 months"), date_labels = "%b") +     # Improving the x-axis to show all months
xlab("Months in 2020") +                                                 # Label for x-axis
scale_y_continuous("No. of people",                                      # Label for y-axis
                     labels = function(x) prettyNum(x, big.mark=",", scientific = FALSE)
                   ) +
ggtitle("COVID-19: Total tesed and total cases (A & B)") +               # Title for the plot
scale_color_identity(name = "",                                          # Legend for the line plot
                     breaks = c("red", "blue"),
                     labels = c("Total tested", "Total confirmed cases"),
                     guide = "legend")
# multiple_grid_plot_theme_config



##########################################################################################################################
####################################### Step 4: Finally make a grid of all 3 plots ######################################
##########################################################################################################################
# options(repr.plot.width=7, repr.plot.height=7)

ggarrange(total_tested_plot, total_confirmed_cases_plot, combined_plot)


-----

-----

# Ploynomial regression

[Source to this shit](https://www.kaggle.com/pedrohenriquecardoso/analyzing-and-predicting-covid-19-updated-05-20)

## Training the model

In [ ]:
set.seed(2020)
# The set.seed() function sets the starting number used to generate a sequence of random numbers – it ensures that you get the same result if you start with that same seed each time you run the same process.


split_training_testing_data <- initial_time_split(denmark_covid_data, prop = 0.8)
training_set <- training(split_training_testing_data)
testing_set <- testing(split_training_testing_data)
trade_off <- tibble(degree = double(), train_rmse = double(), test_rmse = double())

for(d in 1:10){
    polynomial_model <- lm(total_cases ~ poly(day, degree = d, raw = TRUE), training_set)
    training_set["fitted"] <- predict(polynomial_model, training_set)
    testing_set["predicted"] <- predict(polynomial_model, testing_set)
    trade_off <- add_row(
        trade_off,
        degree = d, 
        train_rmse = rmse_vec(training_set$total_cases, training_set$fitted), 
        test_rmse = rmse_vec(testing_set$total_cases, testing_set$predicted)
    )
}

best_degree <- arrange(trade_off, test_rmse, train_rmse)$degree[1]
best_test_rmse <- round(arrange(trade_off, test_rmse, train_rmse)$test_rmse[1],2)
best_train_rmse <- round(arrange(trade_off, test_rmse, train_rmse)$train_rmse[1],2)
polynomial_model <- lm(total_cases ~ poly(day, best_degree, raw = TRUE),training_set)

print(paste("The best degree is: ",best_degree))
print("Ordered Trade off Degree Table - Train vs Test RMSE")
arrange(trade_off,test_rmse,train_rmse,degree)

In [ ]:
ggplot(trade_off,aes(x = degree)) +
geom_vline(xintercept = best_degree, linetype="dashed", color="gray60") +
  geom_point(aes(y = train_rmse), color = "tomato1", size = 4) +
  geom_point(aes(y = test_rmse), color = "cyan4", size = 4) +
  geom_line(aes(y = train_rmse, color = "tomato1"), size = 1.5, alpha = .5) +
  geom_line(aes(y = test_rmse, color = "cyan4"), size = 1.5, alpha = .5) +
  annotate(
    "text",
    x = best_degree - .5,
    y = max(c(trade_off$train_rmse,trade_off$test_rmse)),
    label = "Best RMSE",
    size = 5,
    color = "gray40"
  ) +
  scale_y_continuous("RMSE",
                     labels = function(x) prettyNum(x, big.mark=",", scientific = FALSE)
                     ) +
  labs(
    title = "Train and Test RMSE Trade Off by Polynomial Degree",
    x = "Polynomial Degree",
    y = "RMSE",
    caption = paste(
        "Best Degree: ",
        best_degree,
        ". Best Train RMSE: ", 
        best_train_rmse,
        ". Best Test RMSE: ",
        best_test_rmse
    )
  ) +
  scale_color_identity(
    name = "Line",
    breaks = c("cyan4", "tomato1","gray60"),
    labels = c("Test", "Train", "Best"),
    guide = "legend"
  ) +
  theme(plot.caption = element_text(hjust = .5))

In [ ]:
testing_set["pred"] = predict(polynomial_model, testing_set)

ggplot(testing_set, aes(x = date)) +
  geom_point(aes(y = total_cases, color = "cyan4"), size = 4) +
  geom_line(aes(y = pred, color = "tomato1"), size = 1.5, alpha = .8) +
  labs(
    title = "Test Data Fitted Model",
    x = "Days Since First Case",
    y = "Cases",
    caption = paste("Polynomial degree: ", best_degree,"."," RMSE: ",best_test_rmse)
  ) +
  scale_color_identity(
    name = "Legend",
    breaks = c("cyan4", "tomato1"),
    labels = c("Real", "Fitted"),
    guide = "legend"
  )

In [ ]:
polynomial_model <- lm(total_cases ~ poly(day, best_degree, raw = TRUE), denmark_covid_data)
max_date <- max(denmark_covid_data$date)

new_data <- tibble(
  date = seq(as.Date(max_date+1), by="day", length = 14),
  day = seq(max(denmark_covid_data$day)+1, max(denmark_covid_data$day)+14, by=1),
  total_cases = c(0)
)

covid_final <- select(denmark_covid_data, date, day, total_cases)
covid_final["predicted"] <- predict(polynomial_model, covid_final)
new_data["predicted"] <- predict(polynomial_model, new_data)

rmse <- round(rmse_vec(covid_final$total_cases, covid_final$predicted), 3)
r2 <- round(rsq_vec(covid_final$total_cases, covid_final$predicted)*100, 3)
mae <- round(mae_vec(covid_final$total_cases, covid_final$predicted), 3)

In [ ]:
ggplot() +
  geom_point(data = covid_final, aes(x = date, y = total_cases, color="cyan4"), size = 3) +
  geom_point(data = new_data, aes(x = date, y = predicted, color="tomato1"), size = 3) +
  geom_line(data = covid_final, aes(x = date, y = predicted), size = 1.5, alpha = .5, color="tomato1") +
  labs(
    title = "Polynomial Model - Prediction for the next 3 days",
    subtitle = paste("RMSE: ",rmse," | R2: ",r2, " | MAE: ",mae),
    x = "Days Since First Case",
    y = "Cases",
    caption = paste("Data until: ", max_date,".")
  ) +
  scale_color_identity(
    name = "Legend",
    breaks = c("cyan4", "tomato1"),
    labels = c("Real", "Predicted"),
    guide = "legend"
  )

In [ ]:
print("Fitted Model, last 30 days")
select(covid_final, date, day, total_cases, predicted) %>% mutate(difference = total_cases - predicted) %>% tail(30)
print("Next 2 weeks prediction")
new_data

In [ ]:
tail(denmark_covid_data)

-----

-----

## 5. Prediction

*This is a work in progress*

## 6. Conclusion of EDA
*This is a work in progress*